In [1]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import config


In [2]:
test_transform = transforms.Compose([
    transforms.Resize((config.IMAGE_SIZE, config.IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


In [3]:
test_dataset = datasets.ImageFolder(
    config.TEST_DIR,
    transform=test_transform
)

test_loader = DataLoader(
    test_dataset,
    batch_size=config.BATCH_SIZE,
    shuffle=False
)

print("Test classes:", test_dataset.classes)


Test classes: ['dyslexic', 'normal']


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.resnet50(pretrained=False)

num_features = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(num_features, 256),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(256, 1),
    nn.Sigmoid()
)

model.load_state_dict(torch.load(config.FINETUNED_MODEL_PATH))
model.to(device)
model.eval()

print("✅ Fine-tuned model loaded")


c:\Users\ASUS\anaconda3\envs\dyslexia_env\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\ASUS\anaconda3\envs\dyslexia_env\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


✅ Fine-tuned model loaded


In [7]:
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        preds = (outputs > 0.4).int()


        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())


In [8]:
print("\nAccuracy:", accuracy_score(all_labels, all_preds))

print("\nClassification Report:")
print(classification_report(
    all_labels,
    all_preds,
    target_names=["dyslexic", "normal"]
))

print("\nConfusion Matrix:")
print(confusion_matrix(all_labels, all_preds))



Accuracy: 0.7700421568800381

Classification Report:
              precision    recall  f1-score   support

    dyslexic       0.97      0.67      0.79     37136
      normal       0.60      0.96      0.74     19557

    accuracy                           0.77     56693
   macro avg       0.79      0.82      0.77     56693
weighted avg       0.85      0.77      0.78     56693


Confusion Matrix:
[[24797 12339]
 [  698 18859]]
